<a href="https://colab.research.google.com/github/Fatima-Ebrahim/grocery-sales-data-analysis/blob/main/Grocery_Sales_data_analysis_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
import os
print(uploaded)

{'kaggle.json': b'{"username":"fatimaf2000f","key":"201592298ca14711bca4033accbd8afb"}'}


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d andrexibiza/grocery-sales-dataset

Dataset URL: https://www.kaggle.com/datasets/andrexibiza/grocery-sales-dataset
License(s): CC0-1.0


In [6]:
import zipfile
with zipfile.ZipFile("grocery-sales-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("grocery-sales")

In [9]:
import pandas as pd

# ============================================================
# الخطوة 1: تحميل البيانات من ملفات CSV في مجلد "grocery-sales"
# ============================================================

def load_data():
    file_paths = {
        "المبيعات": "grocery-sales/sales.csv",
        "المنتجات": "grocery-sales/products.csv",
        "العملاء": "grocery-sales/customers.csv",
        "الموظفون": "grocery-sales/employees.csv",
        "الفئات": "grocery-sales/categories.csv",
        "المدن": "grocery-sales/cities.csv",
        "الدول": "grocery-sales/countries.csv"
    }

    dataframes = {}
    for name, path in file_paths.items():
        df = pd.read_csv(path)
        dataframes[name] = df

    return dataframes


# ============================================================
# الخطوة 2: تحليل مبدئي لأي جدول بيانات
# ============================================================

def analyze_data(df, name):
    print(f"\n{'='*60}\n تحليل جدول: {name}\n{'='*60}")

    print(f"\n• عدد الصفوف: {df.shape[0]:,}")
    print(f"• عدد الأعمدة: {df.shape[1]:,}")

    print("\n• أسماء الأعمدة:")
    for col in df.columns:
        print(f"  - {col}")

    print("\n• أنواع البيانات:")
    print(df.dtypes)

    missing = df.isnull().sum()
    if missing.any():
        print("\n• الأعمدة التي تحتوي على قيم مفقودة:")
        print(missing[missing > 0])

        print("\n• النسبة المئوية للقيم المفقودة:")
        print(((missing / len(df)) * 100).round(2)[missing > 0])
    else:
        print("\n✔ لا توجد قيم مفقودة.")

    duplicates = df.duplicated().sum()
    print(f"\n• عدد الصفوف المكررة: {duplicates}")


# ============================================================
# الخطوة 3: تنظيف جدول البيانات
# ============================================================

def clean_data(df):
    original_shape = df.shape

    # 1. تحويل الأعمدة التي تحتوي على تواريخ
    for col in df.columns:
        if 'date' in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # 2. معالجة القيم المفقودة
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['int64', 'float64']:
                df[col] = df[col].fillna(df[col].median())
            elif df[col].dtype == 'datetime64[ns]':
                df[col] = df[col].fillna(df[col].mode()[0])  # مثلاً أكثر تاريخ تكراراً
            else:
                df[col] = df[col].fillna("غير معروف")

    # 3. إزالة التكرارات
    df.drop_duplicates(inplace=True)

    removed = original_shape[0] - df.shape[0]
    print(f" تم تنظيف البيانات ({removed} صفوف مكررة أُزيلت)")
    return df



# ============================================================
# الخطوة 4: توليد تقرير شامل لكل الجداول
# ============================================================

def generate_report(dataframes):
    report_data = []
    for name, df in dataframes.items():
        report_data.append({
            ' الجدول': name,
            ' الصفوف': df.shape[0],
            ' الأعمدة': df.shape[1],
            ' القيم المفقودة': int(df.isnull().sum().sum()),
            ' التكرارات': int(df.duplicated().sum())
        })
    return pd.DataFrame(report_data)


# ============================================================
#  التنفيذ
# ============================================================


dataframes = load_data()

print(f"\n{'*'*60}\n عرض محتويات الجداول قبل التحليل\n{'*'*60}")
for name, df in dataframes.items():
    print(f"\n📄 أول 5 صفوف من جدول: {name}")
    display(df.head())

print(f"\n{'*'*60}\n التحليل المبدئي قبل التنظيف\n{'*'*60}")
for name, df in dataframes.items():
    analyze_data(df, name)


print(f"\n{'*'*60}\n بدء تنظيف البيانات\n{'*'*60}")
for name in dataframes:
    dataframes[name] = clean_data(dataframes[name])


print(f"\n{'*'*60}\n التحليل بعد التنظيف\n{'*'*60}")
for name, df in dataframes.items():
    analyze_data(df, f"{name} (نظيف)")


print(f"\n{'*'*60}\n التقرير النهائي\n{'*'*60}")
final_report = generate_report(dataframes)
display(final_report)





************************************************************
 عرض محتويات الجداول قبل التحليل
************************************************************

📄 أول 5 صفوف من جدول: المبيعات


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3



📄 أول 5 صفوف من جدول: المنتجات


,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,1,Flour - Whole Wheat,74.2988,3,Medium,2018-02-16 08:21:49.190,Durable,Unknown,0.0
1,2,Cookie Chocolate Chip With,91.2329,3,Medium,2017-02-12 11:39:10.970,Unknown,Unknown,0.0
2,3,Onions - Cippolini,9.1379,9,Medium,2018-03-15 08:11:51.560,Weak,False,111.0
3,4,"Sauce - Gravy, Au Jus, Mix",54.3055,9,Medium,2017-07-16 00:46:28.880,Durable,Unknown,0.0
4,5,Artichokes - Jerusalem,65.4771,2,Low,2017-08-16 14:13:35.430,Durable,True,27.0



📄 أول 5 صفوف من جدول: العملاء


...

,CityID,CityName,Zipcode,CountryID
0,1,Dayton,80563,32
1,2,Buffalo,17420,32
2,3,Chicago,44751,32
3,4,Fremont,20641,32
4,5,Virginia Beach,62389,32



📄 أول 5 صفوف من جدول: الدول


,CountryID,CountryName,CountryCode
0,1,Armenia,AN
1,2,Canada,FO
2,3,Belize,MK
3,4,Uganda,LV
4,5,Thailand,VI



************************************************************
 التحليل المبدئي قبل التنظيف
************************************************************

 تحليل جدول: المبيعات

• عدد الصفوف: 6,758,125
• عدد الأعمدة: 9

• أسماء الأعمدة:
  - SalesID
  - SalesPersonID
  - CustomerID
  - ProductID
  - Quantity
  - Discount
  - TotalPrice
  - SalesDate
  - TransactionNumber

• أنواع البيانات:
SalesID                int64
SalesPersonID          int64
CustomerID             int64
ProductID              int64
Quantity               int64
Discount             float64
TotalPrice           float64
SalesDate             object
TransactionNumber     object
dtype: object

• الأعمدة التي تحتوي على قيم مفقودة:
SalesDate    67526
dtype: int64

• النسبة المئوية للقيم المفقودة:
SalesDate    1.0
dtype: float64

• عدد الصفوف المكررة: 0

 تحليل جدول: المنتجات

• عدد الصفوف: 452
• عدد الأعمدة: 9

• أسماء الأعمدة:
  - ProductID
  - ProductName
  - Price
  - CategoryID
  - Class
  - ModifyDate
  - Resistant
  -

,الجدول,الصفوف,الأعمدة,القيم المفقودة,التكرارات
0,المبيعات,6758125,9,0,0
1,المنتجات,452,9,0,0
2,العملاء,98759,6,0,0
3,الموظفون,23,8,0,0
4,الفئات,11,2,0,0
5,المدن,96,4,0,0
6,الدول,206,3,0,0
